In [4]:
# For now, I'm just focusing on delta hedging. After that is done, we make a system of equations
# to set delta and vega to 0 and solve with np.linalg.solve() (see examples/ex6sol for reference).
# %pip install pyfinance
# %pip install py_vollib
# %pip install pandas
# %pip install matplotlib

import pandas as pd
from pyfinance.options import BSM
from datetime import datetime, timedelta
import py_vollib
from py_vollib.black_scholes.implied_volatility import implied_volatility
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.pyplot as plt

df = pd.read_csv("../data/apple.csv")
df = df.drop(["Unnamed: 0"], axis=1)
df['date'] = pd.to_datetime(df['Date'])
df['T'] = (pd.to_datetime('2024-01-19') - df['date']).dt.days / 252 # T = time to expiration in years, 252 trading days
df = df.drop(["date"], axis=1)
df.head()

#interest rate data
df2 = pd.read_csv('../code/csv-data/daily-treasury-rates.csv')
df2 = df2[['Date', '13 WEEKS COUPON EQUIVALENT']]
df2['Date'] = pd.to_datetime(df2['Date'], format='%m/%d/%Y')
df2['Date'] = df2['Date'].dt.strftime('%Y-%m-%d')
df2 = df2[df2['Date'] >= '2023-08-21']
df2['13 WEEKS COUPON EQUIVALENT'] = df2['13 WEEKS COUPON EQUIVALENT'] / 100
df_result = pd.merge(df, df2, on='Date', how='left')
df_result.rename({"13 WEEKS COUPON EQUIVALENT": "Interest rate"}, axis="columns", inplace=True)
df_result['Interest rate'].fillna(df_result['Interest rate'].median(), inplace=True)
df_result.head()

,Date,Underlying,C170,C175,C180,C185,C190,P170,P175,P180,P185,P190,T,Interest rate
0,2023-08-21,175.84,16.40,13.27,10.30,7.85,5.91,6.81,8.65,10.85,13.35,16.50,0.599206,0.0546
1,2023-08-22,177.23,17.50,14.10,11.10,8.51,6.34,6.26,8.10,10.10,12.75,15.50,0.595238,0.0546
2,2023-08-23,181.12,20.20,16.65,13.45,10.55,8.08,5.25,6.71,8.50,10.70,13.33,0.591270,0.0546
3,2023-08-24,176.38,16.97,13.75,10.80,8.35,6.25,6.85,8.62,10.75,13.35,16.60,0.587302,0.0547
4,2023-08-25,178.61,18.50,15.15,11.85,9.19,6.90,5.90,7.50,9.50,11.90,14.81,0.583333,0.0550


# Delta-vega hedging a single option

In [8]:
# Copy-paste spaghetti, refactor
def delta_vega_hedge(option_data, colname):
    initial = option_data.iloc[0]
    option_amount = 1000
    interest_rate = initial['Interest rate']
    option_type = 'call' if colname[0] == "C" else "put"
    stock_price = initial['Underlying']
    option_price = initial[colname]
    strike = int(colname[1:])
    time_to_maturity = initial['T']
    volatility = implied_volatility(price=option_price, S=stock_price, K=strike, t=time_to_maturity, r=interest_rate, flag=option_type[0])

    # Initial stuff at t = 0
    bsm = BSM(kind=option_type, S0=stock_price, K=strike, T=time_to_maturity, r=interest_rate, sigma=volatility)
    delta = bsm.delta()
    gamma = bsm.gamma()
    vega = bsm.vega()
    print(delta)
    print(gamma)
    print(vega)
    

In [9]:
delta_vega_hedge(df_result, "C170")

0.7039443038937685
0.013719648918889866
47.041577446648766
